## Índice

- [1. Introdução](#1.-Introdução)
- [2. Fonte dos dados](#2.-Fonte-dos-dados)
- [3. Possíveis Perguntas](#3.Possíveis-Perguntas)
- [4. Dados](#4.Dados)

## 1. Introdução

<hr />

Qualquer viajante frequente já deve ter se perguntado que rotas as companhias aéreas oferecem a partir do lugar onde ele mora, de modo a poder viajar mais e sobretudo para lugares ainda desconhecidos. Para responder a esta pergunta, pode-se utilizar serviços web de pesquisa de voos, tais como o Skyscanner e o Google Flights, descobrindo a resposta trecho a trecho (ao mesmo tempo em que se verifica o preço da passagem), mas pode-se responder a esta mesma pergunta de forma mais sistemática, através de ferramentas de Data Science. 
A vantagem de sistematizar a informação acerca das rotas supracitadas, através de conjuntos de dados bem consolidados, é ir além das limitações encontradas nas ferramentas web tais quais as descritas anteriormente. Por exemplo:

1) Como garantir que nas ferramentas web existentes estão sendo exibidas todas as opções de rotas?

2) Como buscar voos com Stopover com quantidade de dias indeterminada?

3) Como buscar voos apenas de companhias aéreas que me dão pontos de fidelidade? 
     

## 2. Fonte dos dados


É necessário que se tenha um banco de dados de rotas bem organizado. Felizmente, existem alguns conjuntos de dados deste tipo, por exemplo o disponibilizado pelo OpenFlights, que tem dados disponíveis acerca de:

- **Aeroportos**;    

- **Companhias Aéreas**; e 

- **Rotas de Voo**.
    
Pode-se fazer o download desses dados no endereço https://www.kaggle.com/open-flights/datasets , no qual estão distribuídos os 3 conjuntos de dados supracitados em formato CSV.


## 3. Possíveis Perguntas   

De posse dos conjuntos de dados supracitados, pode-se elencar uma série de possíveis perguntas, tais quais as expostas a seguir:

1) Que países possuem mais companhias aéreas?

2) Que países possuem mais aeroportos?

3) Quais são os aeroportos existentes em uma localidade determinada?

4) Que cidades possuem mais aeroportos?

5) Que aeroportos estão localizados em altitudes extremas?

6) Em que países chegam mais rotas de voos?

7) De quais países saem mais companhias aéreas?

8) Que companhias aéreas possuem mais rotas de voo?

9) Que companhias aéreas vão a mais países?

10) Que rotas possuem a maior quantidade de voos?

11) Que rotas possuem a maior quantidade de companhias aéreas?


## 4. Dados   


In [9]:
import pandas as pd

## Cias Aéreas

airlines = pd.read_csv('data/airlines.csv', na_values='\\N')      # ler airlines
airlines = airlines.loc[airlines['Active'] == 'Y', :].drop(columns='Active')      # remover as desativadas

## Aeroportos, estações de trem e portos

airports = pd.read_csv('data/airports.csv', na_values='\\N', header=None)     # ler airports
airports.rename(columns={0:'Airport ID', 1:'Name', 2:'City', 3:'Country', 4:'IATA', 5:'ICAO', 
                 6:'Latitude', 7:'Longitude', 8:'Altitude', 9:'HoursFromUTC', 10:'DST',
                11:'Timezone', 12:'Type', 13:'Source'}, inplace=True)    
airports = airports.loc[airports['Type'] == 'airport', :].drop(columns=['Type', 'Source'])  # apenas aeroportos

## Rotas

routes = pd.read_csv('data/flights.csv', na_values='\\N')
cols = ['airline ID', 'source airport id', 'destination airport id']
routes = routes[routes['stops'] == 0].dropna(how='any', subset=cols).drop(columns='stops')  # rotas diretas
for c in cols:
    routes.loc[:, c] = routes[c].astype('int')   
    
## Merge de rotas with all the rest
cols = ['Name', 'City', 'Country', 'Latitude', 'Longitude']
df = (pd.merge(routes, airports, left_on='source airport id', right_on='Airport ID', how='inner')
    .rename(columns={c:'source ' + c.lower() for c in cols})
    .drop(airports.columns, axis=1, errors='ignore')
    .merge(airports, left_on='destination airport id', right_on='Airport ID', how='inner')
    .rename(columns={c:'destination ' + c.lower() for c in cols})
    .drop(airports.columns, axis=1, errors='ignore')
    .merge(airlines, left_on='airline ID', right_on='Airline ID', how='inner')
    .rename(columns={'Name': 'airline name'})
    .drop(airlines.columns, axis=1, errors='ignore')
    .sort_index(axis=1)
)